In [1]:
from IsingLib import Model, Formula, DaFormat
from pyqubo import Array, Sum
import numpy as np

np.set_printoptions(threshold=30)
np.set_printoptions(edgeitems=20)
np.set_printoptions(linewidth=200)

In [2]:
city = 5
vartype = 'BINARY'
x = Array.create('x', (city, city), vartype)
dis_li = [[0, 10, 8, 8, 4], [10, 0, 6, 11, 7], [8, 6, 0, 14, 8], [9, 11, 14, 0, 6], [4, 7, 8, 6, 0]]
distance = np.zeros((city**2, city**2))
for i in range(city):
    point = (i + 1) % city
    distance[i*city: (i+1)*city, point*city: (point+1)*city] = dis_li
distance = distance + distance.T
time_const = Sum(0, city, lambda i: (Sum(0, city, lambda j: x[i, j]) - 1) ** 2)
city_const = Sum(0, city, lambda j: (Sum(0, city, lambda i: x[i, j]) - 1) ** 2)

In [3]:
const_model = Model.create(vartype, time_const+city_const)
aim_model = Model(vartype, distance, symbol='x', shape=(city,city))

In [5]:
print(aim_model)

         x[0][0]  x[0][1]  x[0][2]  x[0][3]  x[0][4]  x[1][0]  x[1][1]  \
x[0][0]      0.0      0.0      0.0      0.0      0.0      0.0     10.0   
x[0][1]      0.0      0.0      0.0      0.0      0.0     10.0      0.0   
x[0][2]      0.0      0.0      0.0      0.0      0.0      8.0      6.0   
x[0][3]      0.0      0.0      0.0      0.0      0.0      9.0     11.0   
x[0][4]      0.0      0.0      0.0      0.0      0.0      4.0      7.0   
x[1][0]      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
x[1][1]      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
x[1][2]      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
x[1][3]      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
x[1][4]      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
x[2][0]      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
x[2][1]      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
x[2][2]      0.0      0.0      0.0    

In [6]:
h = Formula(aim=aim_model, const=const_model)

In [7]:
print(h)

H = {'aim': <IsingLib.model.Model object at 0x10f87d2d0>, 'const': <IsingLib.model.Model object at 0x12322bf50>}


In [22]:
model = h.compile(const=10)
print(model)
print(model.const)

         x[0][0]  x[0][1]  x[0][2]  x[0][3]  x[0][4]  x[1][0]  x[1][1]  \
x[0][0]    -20.0     20.0     20.0     20.0     20.0     20.0     10.0   
x[0][1]      0.0    -20.0     20.0     20.0     20.0     10.0     20.0   
x[0][2]      0.0      0.0    -20.0     20.0     20.0      8.0      6.0   
x[0][3]      0.0      0.0      0.0    -20.0     20.0      9.0     11.0   
x[0][4]      0.0      0.0      0.0      0.0    -20.0      4.0      7.0   
x[1][0]      0.0      0.0      0.0      0.0      0.0    -20.0     20.0   
x[1][1]      0.0      0.0      0.0      0.0      0.0      0.0    -20.0   
x[1][2]      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
x[1][3]      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
x[1][4]      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
x[2][0]      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
x[2][1]      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
x[2][2]      0.0      0.0      0.0    

In [9]:
ans = model.solve('sa')

In [10]:
print(ans.states)
print(ans.enegries)

[array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])]
[35.0]


In [12]:
for state in ans.states:
    print(h.check_constraints(state, 'const'))

0.0


In [13]:
# import itertools

# for v in itertools.permutations(range(city)):
#     d = 0
#     for i in range(city):
#         d += dis_li[v[i-1]][v[i]]
#     print(d, v)

In [16]:
da_model = model.to_da()
da = DaFormat(**da_model, api_key='../../../da_key_2.json', version=2)

In [17]:
job_id = da.da_solver()
da.get_job(job_id)

In [18]:
da.get_energy(job_id)

array([-65, -65, -65, -65, -65, -65, -65, -65, -65, -65, -65, -65, -65, -65, -65])

In [19]:
state_li = da.get_configuration(job_id)
for state in state_li:
    print(state)

{'x[0][0]': 0, 'x[0][1]': 0, 'x[0][2]': 0, 'x[0][3]': 0, 'x[0][4]': 1, 'x[1][0]': 0, 'x[1][1]': 0, 'x[1][2]': 0, 'x[1][3]': 1, 'x[1][4]': 0, 'x[2][0]': 0, 'x[2][1]': 1, 'x[2][2]': 0, 'x[2][3]': 0, 'x[2][4]': 0, 'x[3][0]': 0, 'x[3][1]': 0, 'x[3][2]': 1, 'x[3][3]': 0, 'x[3][4]': 0, 'x[4][0]': 1, 'x[4][1]': 0, 'x[4][2]': 0, 'x[4][3]': 0, 'x[4][4]': 0}
{'x[0][0]': 0, 'x[0][1]': 0, 'x[0][2]': 0, 'x[0][3]': 0, 'x[0][4]': 1, 'x[1][0]': 0, 'x[1][1]': 1, 'x[1][2]': 0, 'x[1][3]': 0, 'x[1][4]': 0, 'x[2][0]': 0, 'x[2][1]': 0, 'x[2][2]': 1, 'x[2][3]': 0, 'x[2][4]': 0, 'x[3][0]': 1, 'x[3][1]': 0, 'x[3][2]': 0, 'x[3][3]': 0, 'x[3][4]': 0, 'x[4][0]': 0, 'x[4][1]': 0, 'x[4][2]': 0, 'x[4][3]': 1, 'x[4][4]': 0}
{'x[0][0]': 0, 'x[0][1]': 0, 'x[0][2]': 0, 'x[0][3]': 0, 'x[0][4]': 1, 'x[1][0]': 1, 'x[1][1]': 0, 'x[1][2]': 0, 'x[1][3]': 0, 'x[1][4]': 0, 'x[2][0]': 0, 'x[2][1]': 0, 'x[2][2]': 1, 'x[2][3]': 0, 'x[2][4]': 0, 'x[3][0]': 0, 'x[3][1]': 1, 'x[3][2]': 0, 'x[3][3]': 0, 'x[3][4]': 0, 'x[4][0]': 0, 'x[

In [21]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

model_dwave = h.compile(const=10)
bqm = model_dwave.to_bqm()
# print(tsp_dwave.to_ising())
print(bqm)

sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample(bqm)
print(response.record[0][0].reshape(city, city))

BinaryQuadraticModel({0: -20.0, 1: -20.0, 2: -20.0, 3: -20.0, 4: -20.0, 5: -20.0, 6: -20.0, 7: -20.0, 8: -20.0, 9: -20.0, 10: -20.0, 15: -20.0, 20: -20.0, 21: -20.0, 22: -20.0, 23: -20.0, 24: -20.0, 11: -20.0, 16: -20.0, 12: -20.0, 17: -20.0, 13: -20.0, 18: -20.0, 14: -20.0, 19: -20.0}, {(0, 1): 20.0, (0, 2): 20.0, (0, 3): 20.0, (0, 4): 20.0, (0, 5): 20.0, (0, 6): 10.0, (0, 7): 8.0, (0, 8): 8.0, (0, 9): 4.0, (0, 10): 20.0, (0, 15): 20.0, (0, 20): 20.0, (0, 21): 10.0, (0, 22): 8.0, (0, 23): 9.0, (0, 24): 4.0, (1, 2): 20.0, (1, 3): 20.0, (1, 4): 20.0, (1, 5): 10.0, (1, 6): 20.0, (1, 7): 6.0, (1, 8): 11.0, (1, 9): 7.0, (1, 11): 20.0, (1, 16): 20.0, (1, 20): 10.0, (1, 21): 20.0, (1, 22): 6.0, (1, 23): 11.0, (1, 24): 7.0, (2, 3): 20.0, (2, 4): 20.0, (2, 5): 8.0, (2, 6): 6.0, (2, 7): 20.0, (2, 8): 14.0, (2, 9): 8.0, (2, 12): 20.0, (2, 17): 20.0, (2, 20): 8.0, (2, 21): 6.0, (2, 22): 20.0, (2, 23): 14.0, (2, 24): 8.0, (3, 4): 20.0, (3, 5): 9.0, (3, 6): 11.0, (3, 7): 14.0, (3, 8): 20.0, (3, 9):